# MODELOS DE DECISÃO

### *Exemplo - Deep Blue*
A *Deep Blue* fabrica dois tipos de tinta: tinta para exteriores e tinta para interiores. Na produção da tinta utiliza os materiais A e B. O quadro seguinte apresenta a informação necessária sobre as necessidades de materiais A e B para cada tipo de tinta e a disponibilidade dos factores de produção (valores em toneladas):

|           | Tinta ext.| Tinta int.| Disponibilidade  |
|-----------|-----------|-----------|------------------|
| Material A|     1     |     2     |         6        |
| Material B|     6     |     4     |         24       |
	        

Um estudo de mercado estabeleceu que a quantidade máxima procurada da tinta interior é limitada a 2 toneladas diárias.

O preço de venda da tinta exterior e da tinta interior é de 5.000 €/ton e 4.000 €/ton, respectivamente.

Quantas toneladas de tinta exterior e de tinta interior deve a empresa produzir diariamente para maximizar as vendas?


### Formalização:
 \begin{equation} Max Z = 5x1 + 4x2 \end{equation}
 <p style="text-align:center;">Sujeito a</p>
\begin{equation} 1 x1 + 2 x2 ≤ 6 \end{equation} 
\begin{equation}6 x1 + 4 x2 ≤ 24 \end{equation}
\begin{equation}x2 ≤ 2 \end{equation}
\begin{equation} x1 ,x2 ≥ 0\end{equation}

# Resolução através do programa JuMP
Com o JuMP instalado, para o utilizar o programa JuMP deve escrever-se:

In [1]:
using JuMP


Também precisamos de incluir um pacote Julia que forneça um solucionador apropriado. Queremos usar aqui o HiGHS.Optimizer, que é fornecido pelo pacote HiGHS.jl:

In [2]:
using HiGHS

Embora não seja necessário para resolver o problema, também vou chamar o pacote DataFrames para criar os quadros da análise de sensibilidade.

In [3]:
using DataFrames

### Criar um Modelo
Vamos criar o nosso modelo que eu vou denominar "mod". O JuMP cria problemas de forma incremental no objeto mod, que é o nosso problema. Cria-se agora o modelo passando o otimizador para a função `Model`:

In [4]:
mod = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

### Criar variáveis
As variáveis são criadas utilizando `@variable`.
Deve-se dizer a que modelo deve ser adiconada, denomina-se a variável e estabelecem-se também os seus limites.
A macro cria um novo objeto Julia, x1, no modelo atual. 

In [5]:
@variable(mod,x1>=0)

x1

Podem-se estabelecer limites inferiores e superiores, como o caso de x2:

In [6]:
@variable(mod,0<=x2<=2)

x2

**Nota:** O Jupyter apresenta sempre o resultado da última instrução. Caso não queiram que isso aconteça terminam a última instrução com `";"`.

### Criar a Função Objectivo
A função objectivo é criada com a instrução `@objective`. Mais uma vez deve-se dizer em que modelo é adicionada, se estamos perante um modelo de maximização `Max` ou de minimização `Min` e escrever a função obectivo.

In [7]:
@objective(mod, Max, 5x1+4x2)

5 x1 + 4 x2

**Nota:** O Julia não necessita do símbolo `*` para assumir uma multiplicação.

### Criar Restrições
As restrições são criadas com a instrução `@constraint`. Mais uma vez deve-se dizer em que modelo é adicionada, dar um nome à restrição e escrever a restrição. Cada restrição deve ter uma linha.

In [8]:
@constraint(mod, c1, 1x1+2x2<=6)
@constraint(mod, c2, 6x1+4x2<=24)

c2 : 6 x1 + 4 x2 ≤ 24

**Nota:** Para restrições de igualdade deve colocar-se `==`.

Se quisermos verificar o nosso modelo pode-se utilizar a função `print`.

In [9]:
print(mod)

### Resolver
Para resolver utiliza-se a instrução `optimize!`. O símbolo `!`faz parte do nome. O Júlia tem uma convenção de que as funções que alteram os seus argumentos devem terminar em !.

In [10]:
optimize!(mod)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 6e+00]
  Cost   [4e+00, 5e+00]
  Bound  [2e+00, 2e+00]
  RHS    [6e+00, 2e+01]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
2 rows, 2 cols, 4 nonzeros  0s
Presolve : Reductions: rows 2(-0); columns 2(-0); elements 4(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.4999926288e+00 Ph1: 2(2.5); Du: 1(2.49999) 0s
          2     2.1000000000e+01 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  2.1000000000e+01
HiGHS run time      :          0.00


### Apresentar a solução
Para apresentar a solução utiliza-se `solution_sumary`. A Instrução `verbose=true` serve para apresentar o valor das variáveis.

In [11]:
solution_summary(mod,verbose=true)

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 2.10000e+01
  Objective bound    : 2.10000e+01
  Relative gap       : Inf
  Dual objective value : 2.10000e+01
  Primal solution :
    x1 : 3.00000e+00
    x2 : 1.50000e+00
  Dual solution :
    c1 : -5.00000e-01
    c2 : -7.50000e-01

* Work counters
  Solve time (sec)   : 6.17750e-04
  Simplex iterations : 2
  Barrier iterations : 0
  Node count         : -1


## Análise de Sensibilidade
O seguinte conjunto de instruções permite fazer a análise de sensibilidade. A primeira parte para as variáveis de decisão e a segunda parte para as restrições. Façam copy-paste para os vossos modelos. Cuidado com os nomes do modelo, das variáveis e das restrições.

In [12]:
report = lp_sensitivity_report(mod);

In [13]:
function variable_report(xi) 
    return (
        name = name(xi),
        lower_bound = has_lower_bound(xi) ? lower_bound(xi) : -Inf,
        value = value(xi),
        upper_bound = has_upper_bound(xi) ? upper_bound(xi) : Inf,
        reduced_cost = reduced_cost(xi),
        obj_coefficient = coefficient(objective_function(mod), xi),
        allowed_decrease = report[xi][1],
        allowed_increase = report[xi][2],
    )
end

variable_report (generic function with 1 method)

In [14]:
variable_df = DataFrames.DataFrame(variable_report(xi) for xi in all_variables(mod))

Row,name,lower_bound,value,upper_bound,reduced_cost,obj_coefficient,allowed_decrease,allowed_increase
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,x1,0.0,3.0,Inf,-0.0,5.0,-3.0,1.0
2,x2,0.0,1.5,2.0,-0.0,4.0,-0.666667,6.0


In [15]:
function constraint_report(c::ConstraintRef) 
    return (
        name = name(c),
        value = value(c),
        rhs = normalized_rhs(c),
        slack = normalized_rhs(c) - value(c),
        shadow_price = shadow_price(c),
        allowed_decrease = report[c][1],
        allowed_increase = report[c][2],
    )
    end

constraint_report (generic function with 1 method)

In [16]:
constraint_df = DataFrames.DataFrame(constraint_report(ci) 
    for (F, S) in list_of_constraint_types(mod) 
        for ci in all_constraints(mod, F, S) 
            if F == AffExpr
    )

Row,name,value,rhs,slack,shadow_price,allowed_decrease,allowed_increase
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,c1,6.0,6.0,0.0,0.5,-2.0,0.666667
2,c2,24.0,24.0,0.0,0.75,-4.0,12.0
